#Usando Machine Learning para reconhecer máscaras

##Importando bibliotecas

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import files

##Upload dos arquivos

In [ ]:
importar = files.upload()
print(*importar, sep="\n")

##Lendo imagem com OpenCV

In [ ]:
imagem = cv.imread("")

In [ ]:
type(imagem)

##Mostrando imagem

In [ ]:
def mostrar_imagem(imagem):
  imagem_rgb = cv.cvtColor(imagem, cv.COLOR_BGR2RGB)
  plt.imshow(imagem_rgb)

In [ ]:
mostrar_imagem(imagem)

##Pré-processamento da imagem

In [ ]:
imagemgray = cv.cvtColor(imagem, cv.COLOR_BGR2GRAY)
mostrar_imagem(imagemgray)

##Utilizando CascadeClassifier

In [ ]:
classificador = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
faces = classificador.detectMultiScale(imagemgray)

In [ ]:
imagem_copia = np.array(imagem)

In [ ]:
for x,y,w,h in faces:
  cv.rectangle(imagem_copia, (x,y), (x+w, y+h), (0,255,0), 2)

In [ ]:
mostrar_imagem(imagem_copia)

##Recortando e padronizando imagens com o OpenCV

In [ ]:
imagens_cortadas = list()

In [ ]:
for x,y,w,h in faces:
  face = imagem[y:y+h, x:x+w]
  face = cv.resize(face, (160,160))
  imagens_cortadas.append(face)

In [ ]:
len(imagens_cortadas)

In [ ]:
for img in imagens_cortadas:
  print(img.shape)

In [ ]:
mostrar_imagem(imagens_cortadas[0])

##Salvando conjunto de dados

In [ ]:
def salvar_imagens(imagens, caminho):
  if not os.path.exists(caminho):
    os.mkdir(caminho)
  index = len(os.listdir(caminho))
  for imagem in imagens:
    cv.write(f"{caminho}/{index}.jpg", imagem)
    index += 1

##Modelo de ML

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def carrega_dataframe():
  dados = {
      "ARQUIVO": [],
      "ROTULO": [],
      "ALVO": []
  }

  caminho_com_mascara = "/content/drive/MyDrive/Detector de Máscaras/imagens/com-mascara/"
  caminho_sem_mascara = "/content/drive/MyDrive/Detector de Máscaras/imagens/sem-mascara/"

  com_mascara = os.listdir(caminho_com_mascara)
  sem_mascara = os.listdir(caminho_sem_mascara)

  for arquivo in com_mascara:
    dados["ARQUIVO"].append(f"{caminho_com_mascara}{os.sep}{arquivo}")
    dados["ROTULO"].append(f"Com mascara")
    dados["ALVO"].append(1)

  for arquivo in sem_mascara:
    dados["ARQUIVO"].append(f"{caminho_sem_mascara}{os.sep}{arquivo}")
    dados["ROTULO"].append(f"Sem mascara")
    dados["ALVO"].append(0)

  dataframe = pd.DataFrame(dados)

  return dataframe

In [ ]:
dados = carrega_dataframe()

dados.to_csv("imagens_df.csv")

dados = pd.read_csv("imagens_df.csv")

dados.head()

In [ ]:
def ler_imagens(dados):
  arquivos = dados["ARQUIVO"]
  imagens = list()

  for arquivo in arquivos:
    img = cv.cvtColor(cv.imread(arquivo), cv.COLOR_BGR2GRAY).flatten()
    imagens.append(img)

  dados["IMAGEM"] = imagens

In [ ]:
ler_imagens(dados)

In [ ]:
dados.head()

In [ ]:
X = list(dados["IMAGEM"])
y = list(dados["ALVO"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.99, random_state=13)

In [ ]:
pca = PCA(n_components=30)
pca.fit(X_train)

In [ ]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
parametros = {
    "n_neighbors": [2, 3, 5, 11, 19, 23, 29],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattam", "cosine", "l1", "l2"]
}

knn = GridSearchCV(KNeighborsClassifier(), parametros)

knn.fit(X_train, y_train)

In [ ]:
knn.score(X=X_test, y=y_test)

In [ ]:
predicao = knn.predict(X_test)

In [ ]:
verdadeiros_positivos, falsos_positivos, falsos_negativos, verdadeiros_negativos = confusion_matrix(y_test, predicao).ravel()

In [ ]:
verdadeiros_positivos, verdadeiros_negativos

In [ ]:
falsos_positivos, falsos_negativos

In [ ]:
importar = files.upload()
print(*importar, sep="\n")

In [ ]:
classificador = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def processa_imagem(pca, classificador, imagem):
  img = cv.imread(imagem)
  imagem_cinza = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  faces = classificador.detectMultiScale(img)
  vetores = list()
  cont = 0
  fig = plt.figure(figsize=(10,10))
  for x,y,w,h in faces:
    face_cortada = imagem_cinza[y:y+h, x:x+w]
    fig.add_subplot(3, 3, cont+1)
    plt.imshow(cv.cvtColor(face_cortada, cv.COLOR_BGR2RGB))
    cont+=1
    face_cortada = cv.resize(face_cortada, (160,160))
    vetor = face_cortada.flatten()
    vetores.append(vetor)
  plt.show()
  return vetores

In [ ]:
classes = {
    0: "Sem máscara",
    1: "Com máscara"
}

In [ ]:
vetores = processa_imagem(pca, classificador, "/content/20160816_181754.jpg")
c = knn.predict(pca.transform(vetores))

print(*[classes[e] for e in c], sep=' - ')

##Referências
- [Detector de Máscaras com Python - parte 4](https://www.youtube.com/watch?v=nhBrH6amajg)
- [Detector de Máscaras – Parte 4](https://iaexpert.academy/videos/detector-de-mascaras-parte-4/)
- [Dataset](https://drive.google.com/drive/folders/1ii-rbbC9jcAiILm3HeR-y4HxemdKN1BR)